## SNP effect assesment  
### Bio-informatica 3 - Ronald Wedema
##### Author:           James Gray
##### Student number:   384775

*Single nucleotide polymorphisms* (SNPs), are a common form of mutation that can happen on the genetic level of an organism. Although SNP's can have dentrimental effects for the longlevity of the organism, some can be benign or even benificial in some cases.  
  
In this notebook we try to find out if the effect of a SNP can be predicted using python.

## Choosing the data  
To predict the effect of a SNP, you first have to collect a group of closely related sequences to the gene in question. 
In this example the `BCRA1` gene will be examined in different mamalian species known to carry a similar gene.  
* Homo sapiens  
* Mus musculus  
* Pan troglodytes  
* Sus scrofa  
* Papio anubis  
* Felis catus  
* Mustela putorius furo  
* Macaca fascicularis

## System requirements  
In this notebook the commandline-tool `clustalo` (Clustal Omega) will be used to make a *multiple sequence alignment* (MSA). Please make sure you have installed this program correctly using `conda` or other prefered source.  
Further you should make sure python is installed correctly with the following packages.  

In [23]:
import os
import os.path
import subprocess
import sys
from statistics import mean

from Bio.SubsMat.MatrixInfo import blosum100
from Bio import AlignIO, Seq

## Reading and creating required files  
Two files will be read in this step. One will be the protein family with polypeptide sequences, the other will be a file containing the gene nucleotide sequence. This file will also be split from it's header. This is to more easily index to the right index in the nucleotide sequence.


In [24]:
# Online aquired files
protein_group_path = './data/BRCA1_refseq_protein.fasta'
gene_sequene_path = './data/BCRA1.fasta'

with open(protein_group_path) as fp:
    protein_group_multifasta = fp.read()

with open(gene_sequene_path) as fp:
    # Save as a list
    gene_sequence_file = fp.read().split('\n')
    # Get the header
    gene_sequence_header = str(gene_sequence_file[0])
    # Concatonate the sequence in a string
    gene_sequence = "".join(gene_sequence_file[1:-1])

## Creating a SNP on a selected spot in the gene sequence  
Lets say we want to make a mutation at the following position

### Intruduced mutations  
`position` is a 0-based position where you want the mutation to start.  
`original` is to make sure you are making the mutation at the right position in the sequence.  
`mutation` is the desired mutation you want to apply to the nucleotide sequence.  
The only characters that will be inserted are `ACTG` other characters will be handled as a deletion.

In [25]:
position = 3
original = 'GAT'
# Mark a deletion as '-'
mutation = 'GAC'

### Function to introduce the mutation in the sequence

In [26]:
def mutate(position, original, mutation, nuc_list):
    # If the length of the mutation is longer or equal to the original seq
    if len(original) <= len(mutation):
        del nuc_list[position:position+len(mutation)]
        nuc_list.insert(position, mutation)
        nuc_list = list("".join(nuc_list))
    else:
        del nuc_list[position:position+len(original)]
        nuc_list.insert(position, mutation)
        nuc_list = list("".join(nuc_list))

    # Concatonate in list and remove deletions
    for char in range(len(nuc_list)):
        if nuc_list[char] not in 'ACTG':
            nuc_list[char] = ''
    return nuc_list

In [27]:
nuc_list = list(gene_sequence)

message = f"""User specified original sequence doesn't correspond with the actual readings\n
FASTA header:\n{gene_sequence_header}
position [{position}:{position+len(original)}]
user specified:{list(original)}
actual readings:{nuc_list[position:position+len(list(original))]}"""

if not mutation:
    sys.exit('No mutation set in parameters...')

# check if proclaimed original sequence corresponds with actal readings
if list(original) == nuc_list[position:position+len(list(original))]:
    nuc_list = mutate(position, original, mutation, nuc_list)
else:
    print(message)
    message = input('Ignore this? y/n :')
    if message.upper() == 'Y':
        nuc_list = mutate(position, original, mutation, nuc_list)
    else:
        sys.exit('Program shut down...')


mutated_sequence = ''.join(nuc_list)
print('SNP introduced to sequence')

SNP introduced to sequence


### Translate transcipt to polypeptide  
Now we have introduced a SNP to the gene sequence, we need to translate the transcript to a polypeptide sequence. This is going to be our first glimpse into seeing if the SNP applied is going to be malignant or benign.  
  
We will save our mutated sequence in a `Seq()` object from biopython. This is handy, because this object contains a method to do the translation for us. We can compare the length of the transcript of the mutated sequence to the unmutad sequence and see how much they differ.

In [28]:
org_seq = Seq.Seq(gene_sequence)
mut_seq = Seq.Seq(mutated_sequence)

org_seq_translated = org_seq.translate()
mut_seq_translated = mut_seq.translate()

percentage_loss = None

# If the length of the original- and the mutated sequence aren't the same
if len(org_seq_translated) != len(mut_seq_translated):
    print("There is a difference in transcript length...")
    percentage_loss = 100 - (len(org_seq_translated) / len(mut_seq_translated) * 100 )
    print(f"There's a loss of: {percentage_loss}%")


## Write the mutated sequences to the group of proteins to be aligned.  
In order to assess the mutagenity of a SNP, the mutation has to be put into perspective to homologue protein sequences to calculate the conservation.  
  
To make it easy to find back the mutation that's been incorperated into the genome, the original sequence will also be added to the file.

In [29]:
file_out = "./data/SNP_Included.fasta"

data = protein_group_multifasta + '\n'
data += ">OG_Sequence\n"
data += str(org_seq_translated) + '\n\n'
data += f">SNP_Mutation: {original} to {mutation} @ {position}\n"
data += str(mut_seq_translated)

with open(file_out, 'w') as f_out:
    f_out.write(data)

print('combined file written!')

combined file written!


Now the file are combined into one, the MSA can be perfomed. This step will be done using the `subprocess` command under the `sys` package. Again, make sure `clustalo` is installed correctly and added to the systems `PATH`.

In [30]:
alignment_name = "./data/MSA_SNP.aln"
command = f'clustalo --in={file_out} --out={alignment_name} --fmt=clustal'

if os.path.exists(alignment_name):
    # Remove old alignment if the user hasn't set a new filename
    try:
        os.remove(alignment_name)
    except:
        print('Reload the notebook please')

align = subprocess.Popen(["clustalo","-i", file_out, "-o", alignment_name, "--outfmt=clustal"], stdout=subprocess.PIPE)
output = align.communicate()[0]
print('Sequences aligned.')

Sequences aligned.


## Reading and analysing the MSA  
Now the multiple sequence alignment has been generated, we have to read in the file. To do this I've chosen to use the `Bio.AlignIO` package, because it handles `clustal` format alignments.

In [31]:
try:
    msa = AlignIO.read(handle=alignment_name, format='clustal')
except FileNotFoundError:
    print('ERROR: Check if clustalo is correctly installed.')

## The MSA object  
After reading the alignment file into the newly created `msa` instance, we can examine the contents.

In [32]:
print(f'Length of MSA: {len(msa)} rows')
for record in msa:
    print(f'record name: {record.id} \t length: {len(record.seq)} \t first : {record.seq[0:15]}\tlast : {record.seq[-15:]}')

Length of MSA: 10 rows
record name: NP_009225.1 	 length: 1916 	 first : ----------MDLSA	last : PHSHY----------
record name: NP_033894.3 	 length: 1916 	 first : ----------MDLSA	last : TCD---SSEPQDSND
record name: NP_001038958.1 	 length: 1916 	 first : ----------MDLSA	last : PHSHY----------
record name: XP_005657013.1 	 length: 1916 	 first : ----------MDLSV	last : PQSCC---RPCT---
record name: NP_001289032.1 	 length: 1916 	 first : ----------MDLSA	last : PHSHY----------
record name: XP_019673492.1 	 length: 1916 	 first : ----------MDLSA	last : PRATADLSQPCV---
record name: XP_004772665.1 	 length: 1916 	 first : MPQNSLEQEEMDLPA	last : PRAAAHSSQPCV---
record name: NP_001288295.1 	 length: 1916 	 first : ----------MDLSA	last : PHSHY----------
record name: OG_Sequence 	 length: 1916 	 first : ----------MDLSA	last : PHSHY*---------
record name: SNP_Mutation: 	 length: 1916 	 first : ----------MDLSA	last : PHSHY*---------


## Does the translation make any sense?  
As can be seen in the results above, the sequences start with a *methionine* residue. This is logical, because in both eukaryotes and archeae this codon is at the start of every polypeptide sequence. Let's asume that this is a sign that the transcript at least produces a translatable polypeptide. Let's also asume that if the polypeptide **doesn't** start with *methionine* the transcript is going to be garbage.  
  
This chunk of code checks if all the first residues are actually methionine

In [33]:
is_garbage = False
first_residue = "".join([r[0] for r in [x.strip('-') for x in [record.seq for record in msa]]])
if not first_residue == len(first_residue) * first_residue[0] and first_residue[0] == 'M':
    print('First residues are not all Methionine')
    is_garbage = True
else:
    print('First residues start with Methionine')

First residues start with Methionine


## Testing for silent mutations  
Due to the wobble-effect in the last nucleotide of a triplet, the translated amino acid will be the same.  
Example:  
 * `ATA` --> `I`  
 * `ATC` --> `I`  
A SNP changing an `A` to a `C` in this case will cause a mutation in the translated polypeptide sequence.  
  
To check if the SNP applied in the transcript causes a silent mutation a simple comparison of equality will surfice.

In [34]:
og_alignment = msa[-2]
mutated_alignment = msa[-1]
silent_mutation = False

if og_alignment.seq == mutated_alignment.seq:
    print('The sequences appear to be the same')
    silent_mutation = True
else:
    print('The sequence is changed due to the SNP')
    print('Mutation is not silent')

The sequences appear to be the same


## Compare the introduced mutation to the MSA  
A MSA is a way to show how proteins relate to each other. The introduced mutation might be in a site with a very high measure of conservation. Here a misplaced residue will stick out like a splatter of paint on a white canvas. On other sites in the MSA, the residue might not stick out initially, but if the characteristics of the amino acids are further inspected there might be some discrapancies.  
Some of these characteristics are:  
* Acidity  
* Sollubilaty  
* Structure  
* Weight  
* Size  
  
  
Sometimes a risidue is simply interchangable with an other and it might not produce any structural and functional changes in the polypeptide. Again keeping the MSA as a reference will give insight if this is the case. The next section of this notebook is deticated to identifying and scoring these kind of SNP's  
  
## Calculating the average substitution score globally and per site


In [35]:
snp_i = []
is_pointmutation = False
is_frameshift = False
substitution_AA = []
original_AA = []
site_alignment = []

# Find the SNP in sequence if the sequence makes sense
if not is_garbage and not silent_mutation:
    for i, chars in enumerate(zip(mutated_alignment.seq, og_alignment.seq)):

        if chars[0] != chars[1]:
            snp_i.append(i)
            substitution_AA.append(chars[0])
            original_AA.append(chars[1])
            site_alignment.append(msa[0:len(msa)-1, i])


# Calculate averate substitution score of snp/original ~ MSA

if len(snp_i) == 1:
    is_pointmutation = True
    print('Mutation is point mutation')
elif percentage_loss:
    is_frameshift = True
    print('Mutation is a frameshift mutation')


Let's also compare the substituion to the rest of the MSA. While doing this, we'll compare the original residue's average substitution score to the snp's substitution score. Afterwards this will be averaged out.

In [36]:
all_site_scores = []
site_score_org = []
site_score_snp = []
min_conservation = -7
max_conservation = 11

print('Minimum conservation', min_conservation)
print('Maximum conservation', max_conservation)


# Calculate conservation and change in conservation per snp
for site, org, snp in zip(site_alignment, original_AA, substitution_AA):
    site_avg_org = []
    site_avg_snp = []
    for s in site:
        # I'm not really sure what the penalty should be if the site contains gaps and a AA is substituted.
        # So the penalty will be the consithered minimum conservation score
        if s in '*-' or org in '*-' or snp in '*-':
            pass
        else:
            try:
                site_avg_org.append(blosum100[(s, org)])
            except KeyError:
                site_avg_org.append(blosum100[(org, s)])
            try:
                site_avg_snp.append(blosum100[(s, snp)])
            except KeyError:
                site_avg_snp.append(blosum100[(snp, s)])

    if site_avg_org:
        site_score_org.append(mean(site_avg_org))
    if site_avg_snp:
        site_score_snp.append(mean(site_avg_snp))


# for org_s, snp_s, i in zip(site_score_org, site_score_snp, snp_i):
#     print(f'index: {i}\naverage substitution score original~alignment:\t{org_s}\n\
# average substitution score snp~alignment:\t{snp_s}')


Minimum conservation -7
Maximum conservation 11


## Creating the score  
Let's assume that the avarage substitution score of the MSA is a baseline for conservation. The highter the average substitutionscore is to itself is the amount of conservation. For example a `W --> W` substitution is heavily favoured in the `blosum100` and is awarded the maximum score: `11`. On the other hand there is the least favourable substitution, which is `W --> D`. This substitution is penalized the minimum score of `-7`.  
  
If this is these two substitution are the min and max boundries we can set up a scoring system where 1 is the most neutral and 10 is the most deleterous.  
  



$score(siteAvgSNP) = \frac{siteAvgSNP + abs(min(conservation))} {max(conservation) + abs(min(conservation))} * 9 + 1$  
  

## Scoring the SNP



In [37]:
def score(site_avg_snp, min_conservation, max_conservation):
    return (1-(site_avg_snp+abs(min_conservation))/(max_conservation + abs(min_conservation)))*9+1

if is_garbage:
    print(10)
elif silent_mutation:
    print(1)
elif is_frameshift:
    print(10 - (10*(-1*percentage_loss)))
elif is_pointmutation:
    print(score(site_score_snp[0], min_conservation, max_conservation))
else:
    # Mutation placed in gap
    print(1)

1
